In [ ]:
import os, sys

sys.path.append(os.path.dirname(os.path.dirname(os.path.abspath("__file__"))))
from nbafuns import *
from nba_api.stats.endpoints import leaguedashplayerstats

data_DIR = "../fdata/"
fig_DIR = "../figs/player_analysis/"

In [ ]:
theme_idv = themes.theme_xkcd(base_size=12)
theme_idv += theme(
    plot_title=element_text(face="bold", size=16),
)

In [ ]:
dfa = []
for year in range(2004, 2024):
    df1 = pd.read_parquet(
        data_DIR + "NBA" + "_Player_BoxScores_" + "Indv" + "_" + str(year) + ".parquet"
    )
    df1["Season"] = year + 1
    df1["Games_Played"] = len(df1["GAME_ID"].unique())
    dfa.append(df1)
df = pd.concat(dfa)
df["Win"]  = df["WL"] == "W"
df["Loss"] = df["WL"] == "L"

In [ ]:
pts = 30
df_n = df.query(f"PTS>={pts}")

In [ ]:
# df_n.iloc[0]

In [ ]:
# df_n.tail()

In [ ]:
df_p = df_n.groupby(["Season", "Games_Played"])[["PTS", "FGA", "FTA","Win","Loss"]].agg(
    {"PTS": ["count", "sum"], "FGA": ["sum"], "FTA": ["sum"], "Win": ["sum"], "Loss": ["sum"]}
)
df_p.columns = ["Games", "PTS", "FGA", "FTA", "Wins", "Losses"]
df_p = df_p.reset_index()
df_p["Games"] = (1230 * df_p["Games"] / df_p["Games_Played"]).astype(int)
df_p["Wins"] = (1230 * df_p["Wins"] / df_p["Games_Played"]).astype(int)
df_p["TS"] = (df_p["PTS"] / (2 * (df_p["FGA"] + 0.44 * df_p["FTA"]))).round(3)
df_p["Win_p"] = (df_p["Wins"]/df_p["Games"]).round(3)

In [ ]:
df_p["Games_fill"] = "Games"
df_p["Wins_fill"] = "Wins"

In [ ]:
df_p.tail()

In [ ]:
from plotnine import scale_fill_manual, geom_text

In [ ]:
p = (
    ggplot(df_p)
    + aes(x="Season", y="Games")
    + geom_line(group=1)
    + geom_point()
    + geom_bar(aes( fill="Games_fill"),stat="identity", alpha=0.5)
    + geom_bar(aes(fill="Wins_fill", y="Wins"),stat="identity", alpha=0.5)
    # + geom_text(aes(label = "Win_p", y="Wins"),size=6,nudge_y=-20,format_string ="{:0.2f}")
    + scale_fill_manual(values = ["grey","black"])
    + scale_x_continuous(breaks=[2005, 2008, 2012, 2016, 2020, 2024])
    + theme_idv
    + theme(
        plot_caption=element_text(ha="left"),
        legend_title=element_blank(),
        legend_position = [0.22,0.85],
    )
    + labs(
        title=f"Number of {pts} Point Games in a Season",
        caption="Number of Games Scaled to 82 Game Season| @SravanNBA | source:nba.com/stats",
        y=f"{pts} Point Games",
    )
)
p.save(fig_DIR + f"Point_Games_{pts}_seasons_1.png", dpi=300)
p.draw()

In [ ]:
theme_idv = themes.theme_xkcd(base_size=12)
theme_idv += theme(
    plot_title=element_text(face="bold", size=16),
)

In [ ]:
p = (
    ggplot(df_p)
    + aes(x="Season", y="TS")
    + geom_smooth(se=False)
    + geom_point()
    + scale_x_continuous(breaks=[2005, 2008, 2012, 2016, 2020, 2024])
    + theme_idv
    + theme(plot_caption=element_text(ha="left"))
    + labs(
        title=f"True Shooting % of {pts} Point Games in a Season",
        caption="@SravanNBA | source:nba.com/stats",
        y=f"True Shooting %",
    )
)
p.save(fig_DIR + f"Point_Games_{pts}_TS_seasons.png", dpi=300, verbose= False)
p.draw()

In [ ]:
p = (
    ggplot(df_p)
    + aes(x="Season", y="Win_p")
    + geom_smooth(se=False,method ="lm")
    + geom_point()
    + scale_x_continuous(breaks=[2005, 2008, 2012, 2016, 2020, 2023])
    + theme_idv
    + theme(plot_caption=element_text(ha="left"))
    + labs(
        title=f"Win % of {pts} Point Games in a Season",
        caption="@SravanNBA | source:nba.com/stats",
        y=f"Win %",
    )
)
p.save(fig_DIR + f"Point_Games_{pts}_Winp_seasons.png", dpi= 300)
p.draw()